This notebook is for testing auto_arima method in Python.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot

In [2]:
total = pd.read_csv("weekBA.csv", sep=";", delimiter=None, header='infer')

C:\Users\qiong\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,12,13,22,23,46,50,64,67,68,69,70,71,72,73,74,75,76,77,80,84,86) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
total['Date'] = pd.to_datetime(total['CreatedAt'].apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y')))
time_series_df = total.groupby('Date').size().reset_index(name='Count')
time_series_df.head()

,Date,Count
0,2018-01-15,3
1,2018-01-16,6
2,2018-01-17,4
3,2018-01-18,6
4,2018-01-19,4


In [4]:
time_series_df.Date = pd.to_datetime(time_series_df.Date, format='%Y-%m-%d')
time_series_df.head()

,Date,Count
0,2018-01-15,3
1,2018-01-16,6
2,2018-01-17,4
3,2018-01-18,6
4,2018-01-19,4


In [5]:
time_series_df.tail(4)

,Date,Count
743,2020-01-30,67
744,2020-01-31,72
745,2020-02-01,24
746,2020-02-02,26


In [6]:
time_series_df = time_series_df.set_index('Date')
time_series = time_series_df['Count'].resample("W").sum()

In [7]:
time_series

Date
2018-01-21     28
2018-01-28     41
2018-02-04     68
2018-02-11     92
2018-02-18     59
             ... 
2020-01-05    363
2020-01-12    474
2020-01-19    476
2020-01-26    386
2020-02-02    412
Freq: W-SUN, Name: Count, Length: 107, dtype: int64

In [8]:
from pmdarima.arima import auto_arima
stepwise_model = auto_arima(time_series, start_p=1, start_q=1, test='adf',
                           max_p=2, max_q=2, m=52,
                           start_P=0, seasonal=True,
                           d=None, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
stepwise_model.summary()

Fit ARIMA: order=(1, 2, 1) seasonal_order=(0, 1, 1, 52); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 2, 0) seasonal_order=(0, 1, 0, 52); AIC=600.274, BIC=604.215, Fit time=0.221 seconds
Fit ARIMA: order=(1, 2, 0) seasonal_order=(1, 1, 0, 52); AIC=591.801, BIC=599.682, Fit time=7.070 seconds
Fit ARIMA: order=(0, 2, 1) seasonal_order=(0, 1, 1, 52); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 2, 0) seasonal_order=(0, 1, 0, 52); AIC=590.042, BIC=595.953, Fit time=0.740 seconds
Fit ARIMA: order=(1, 2, 0) seasonal_order=(0, 1, 1, 52); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 2, 0) seasonal_order=(1, 1, 1, 52); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(2, 2, 0) seasonal_order=(0, 1, 0, 52); AIC=574.720, BIC=582.601, Fit time=1.591 seconds
Fit ARIMA: order=(2, 2, 1) seasonal_order=(0, 1, 0, 52); AIC=563.404, BIC=573.256, Fit time=2.099 seconds
Fit ARIMA: order=(2, 2, 1) seasonal_order=(1, 1, 0, 52); AIC=563.496, BIC=575.318, Fit

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  107
Model:             SARIMAX(2, 2, 1)x(0, 1, 0, 52)   Log Likelihood                -276.702
Date:                            Tue, 04 Feb 2020   AIC                            563.404
Time:                                    14:31:05   BIC                            573.256
Sample:                                         0   HQIC                           567.193
                                            - 107                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.4711      1.484      0.318      0.751      -2.437       3.379
ar.L1         -0.2637      0.147     -1.794      0.073      -0.552       0.024
ar.L2         -0.3526      0.139     -2.542      0.011      -0.624      -0.081
ma.L1         -0.7878      0.105     -7.539      0.000      -0.993      -0.583
sigma2      1932.8164    407.406      4.744      0.000    1134.315    2731.318
===================================================================================
Ljung-Box (Q):                       30.18   Jarque-Bera (JB):                 4.75
Prob(Q):                              0.87   Prob(JB):                         0.09
Heteroskedasticity (H):               1.06   Skew:                            -0.64
Prob(H) (two-sided):                  0.90   Kurtosis:                         3.70
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [9]:
predict = stepwise_model.predict(n_periods = 1)
predict

array([413.88464343])